In [1]:
with open('data/input_21.txt') as fh:
    file_input = fh.read().strip()

In [19]:
for i, line in enumerate(file_input.split('\n')[1:]):
    print("{} {}".format(i, line))

0 seti 123 0 5
1 bani 5 456 5
2 eqri 5 72 5
3 addr 5 1 1
4 seti 0 0 1
5 seti 0 6 5
6 bori 5 65536 4
7 seti 13431073 4 5
8 bani 4 255 3
9 addr 5 3 5
10 bani 5 16777215 5
11 muli 5 65899 5
12 bani 5 16777215 5
13 gtir 256 4 3
14 addr 3 1 1
15 addi 1 1 1
16 seti 27 9 1
17 seti 0 1 3
18 addi 3 1 2
19 muli 2 256 2
20 gtrr 2 4 2
21 addr 2 1 1
22 addi 1 1 1
23 seti 25 4 1
24 addi 3 1 3
25 seti 17 8 1
26 setr 3 4 4
27 seti 7 7 1
28 eqrr 5 0 3
29 addr 3 1 1
30 seti 5 9 1


In [111]:
def gen_next(five):
    four = 65536 | five
    five = 13431073
    while four >= 1:
        # print(four, five)
        five += four & 255
        five &= 16777215
        five *= 65899
        five &= 16777215
        four >>= 8
    
    return five
    

In [45]:
class VM(object):
    def __init__(self, bound=None, debug=False, maxit = -1):
        self.ip = 0
        self.reg = [0, 0, 0, 0, 0, 0]
        self.bound = bound
        self.mem = []
        self.debug = debug
        self.maxit = maxit
        
    def operations(self, ins, *args):
        ops = {
            'addr': lambda a, b: (self.reg[a] + self.reg[b]),
            'addi': lambda a, b: (self.reg[a] + b),
            'mulr': lambda a, b: (self.reg[a] * self.reg[b]),
            'muli': lambda a, b: (self.reg[a] * b),
            'banr': lambda a, b: (self.reg[a] & self.reg[b]),
            'bani': lambda a, b: (self.reg[a] & b),
            'borr': lambda a, b: (self.reg[a] | self.reg[b]),
            'bori': lambda a, b: (self.reg[a] | b),
            'setr': lambda a, b: (self.reg[a]),
            'seti': lambda a, b: (a),
            'gtir': lambda a, b: (1 if a > self.reg[b] else 0),
            'gtri': lambda a, b: (1 if self.reg[a] > b else 0),
            'gtrr': lambda a, b: (1 if self.reg[a] > self.reg[b] else 0),
            'gtrr_mod': lambda a, b: (1 if self.reg[a]**2 > self.reg[b] else 0),
            'eqir': lambda a, b: (1 if a == self.reg[b] else 0),
            'eqri': lambda a, b: (1 if self.reg[a] == b else 0),
            'eqrr': lambda a, b: (1 if self.reg[a] == self.reg[b] else 0),
        }
        if ins == '#ip':
            self.bound = args[0]
        else:
            self.reg[args[2]] = ops[ins](args[0], args[1])
        
        
    def load(self, inp):
        inp = inp.split('\n')
        self.mem = []
        self.ip = 0
        for line in inp:
            args = line.split()
            if len(args) == 4:
                self.mem.append((args[0], int(args[1]), int(args[2]), int(args[3])))
            elif len(args) == 2:
                pass
#                 self.mem.append((args[0], int(args[1]), 0, 0))
    
    def exe(self):
        if self.bound is not None:
            self.reg[self.bound] = self.ip
        ins = self.mem[self.ip]
        before = (self.ip, self.reg[:])
        self.operations(*ins)
        if self.bound is not None:
            self.ip = self.reg[self.bound]
        self.ip += 1
        if self.debug and before[0] in self.debug:
            print("{:50s} {:20s} {} {}".format(before, ins, (self.ip, self.reg), self.i))
    
    def run(self):
        self.i = 0
        while True:
            self.exe()
            if self.ip < 0 or self.ip >= len(self.mem):
                break
            self.i+=1
            if self.i == self.maxit:
                break

In [58]:
%%time
vm = VM(bound=1, debug=[28], maxit=200000)
vm.load(file_input)
vm.reg[0] = 3115806
#vm.mem[9] = ('gtrr_mod', 2, 4, 3)
vm.run()
print(vm.i, vm.reg)


(28, [3115806, 28, 1, 1, 1, 3115806])              ('eqrr', 5, 0, 3)    (29, [3115806, 28, 1, 1, 1, 3115806]) 1846
(1847, [3115806, 30, 1, 1, 1, 3115806])
CPU times: user 17.7 ms, sys: 4.28 ms, total: 22 ms
Wall time: 18.9 ms


In [48]:
%%time
vm = VM(bound=1, debug=list(range(100)), maxit=20)
vm.load(file_input)
vm.reg[0] = 0
# vm.mem[9] = ('gtrr_mod', 2, 4, 3)
vm.run()
print(vm.i, vm.reg)

(0, [0, 0, 0, 0, 0, 0])                            ('seti', 123, 0, 5)  (1, [0, 0, 0, 0, 0, 123]) 0
(1, [0, 1, 0, 0, 0, 123])                          ('bani', 5, 456, 5)  (2, [0, 1, 0, 0, 0, 72]) 1
(2, [0, 2, 0, 0, 0, 72])                           ('eqri', 5, 72, 5)   (3, [0, 2, 0, 0, 0, 1]) 2
(3, [0, 3, 0, 0, 0, 1])                            ('addr', 5, 1, 1)    (5, [0, 4, 0, 0, 0, 1]) 3
(5, [0, 5, 0, 0, 0, 1])                            ('seti', 0, 6, 5)    (6, [0, 5, 0, 0, 0, 0]) 4
(6, [0, 6, 0, 0, 0, 0])                            ('bori', 5, 65536, 4) (7, [0, 6, 0, 0, 65536, 0]) 5
(7, [0, 7, 0, 0, 65536, 0])                        ('seti', 13431073, 4, 5) (8, [0, 7, 0, 0, 65536, 13431073]) 6
(8, [0, 8, 0, 0, 65536, 13431073])                 ('bani', 4, 255, 3)  (9, [0, 8, 0, 0, 65536, 13431073]) 7
(9, [0, 9, 0, 0, 65536, 13431073])                 ('addr', 5, 3, 5)    (10, [0, 9, 0, 0, 65536, 13431073]) 8
(10, [0, 10, 0, 0, 65536, 13431073])               ('bani', 5, 16777215,

In [109]:
vm = VM(bound=1, debug=[28], maxit=200000)
vm.load(file_input)
vm.reg[0] = 0
# vm.mem[9] = ('gtrr_mod', 2, 4, 3)
vm.run()
print(vm.i, vm.reg)

(28, [0, 28, 1, 1, 1, 3115806])                    ('eqrr', 5, 0, 3)    (29, [0, 28, 1, 0, 1, 3115806]) 1846
(28, [0, 28, 1, 1, 47, 3741829])                   ('eqrr', 5, 0, 3)    (29, [0, 28, 1, 0, 47, 3741829]) 87417
(28, [0, 28, 1, 1, 57, 3052285])                   ('eqrr', 5, 0, 3)    (29, [0, 28, 1, 0, 57, 3052285]) 190173
(200000, [0, 24, 0, 1402, 3117821, 12144778])


Part A

In [119]:
gen_next(0)

3115806

Part B

In [118]:
%%time
n = 0
seen = []
while True:
    n = gen_next(n)
    if n in seen:
        break
    seen.append(n)
print (seen[-1])

13959373
CPU times: user 1.01 s, sys: 19.5 ms, total: 1.03 s
Wall time: 1.09 s
